In [4]:
import os
import sys

# --- 1. CONFIGURATION ---
REPO_URL = "https://github.com/talspec2/red-string.git"
PROJECT_DIR = "red-string"

# --- 2. CLONE CODE FROM GITHUB ---
if not os.path.exists(PROJECT_DIR):
    print(f"🚀 Cloning {PROJECT_DIR} from GitHub...")
    !git clone $REPO_URL
else:
    print(f"🔄 Updating {PROJECT_DIR}...")
    !cd $PROJECT_DIR && git pull

# --- 3. INSTALL DEPENDENCIES FROM REQUIREMENTS.TXT ---
print("📦 Installing dependencies from requirements.txt...")
%pip install -r $PROJECT_DIR/requirements.txt -q

# --- 4. SETUP PATHS ---
if os.path.abspath(PROJECT_DIR) not in sys.path:
    sys.path.append(os.path.abspath(PROJECT_DIR))

🔄 Updating red-string...
Already up to date.
📦 Installing dependencies from requirements.txt...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:

from unsloth import is_bfloat16_supported
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments

from src.config import SYSTEM_INSTRUCTION, MAX_SEQ_LENGTH
from src.data_utils import process, prompt
from src.model_utils import load_model

In [ ]:
raw_data = process(SYSTEM_INSTRUCTION, amount=20000)
dataset = Dataset.from_list(raw_data)
print(f"✅ Data processed. {len(dataset)} examples ready.")

TypeError: process() got an unexpected keyword argument 'prompt'

In [ ]:
model, tokenizer = load_model()
print("✅ Model loaded and adapters attached.")

In [ ]:
dataset = dataset.map(lambda x: prompt(x, tokenizer), batched=True)

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        num_train_epochs=1,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

print("Starting Training")
trainer.train()
print("Done!")